# AIsteroid
[http://bit.ly/aisteroid](http://bit.ly/aisteroid)

In [1]:
from aisteroid import *
%matplotlib nbagg

## Task: Perform astrometry of images

### Choose the image set

In [2]:
if QIPY:
    #listImages() ##See the list of imagesets
    #CONF.SET="example" ##Choose your preferred imageset
    #CONF.CFG="example" ##You choose your preferred observatory configuration (example.cfg)
    CONF.OVERWRITE=1 ##Overwrite all previous actions
    CONF.VERBOSE=1 ## Show all outputs

#### DO NOT TOUCH IF YOU ARE NOT SURE

In [12]:
#DO NOT MODIFY THIS LINES
print0("*"*60+"\nASTROMETRY OF IMAGE SET'%s'\n"%CONF.SET+"*"*60)
OUT_DIR=CONF.SCR_DIR+CONF.SET+"/"
CFG=[line.rstrip('\n') for line in open(CONF.SETS_DIR+CONF.CFG+".cfg")]
AIA=dict()
AIA_FILE=OUT_DIR+CONF.SET+".aia"
SET_FILE=CONF.SETS_DIR+CONF.SET+".zip"
PLOT_DIR=OUT_DIR+"plots/"
FLOG=open(OUT_DIR+"astrometry.log","a")
SYSOPTS=dict(qexit=[True,FLOG])
if not os.path.isfile(AIA_FILE):
    error("Set '%s' has not been unpacked"%CONF.SET)
else:
    System("cp "+CONF.INPUT_DIR+"analysis/* "+OUT_DIR)
    AIA=pickle.load(open(AIA_FILE,"rb"))
    images=AIA["images"]
    sources=AIA["sources"]
    borders=AIA["borders"]
    nimgs=len(images)

************************************************************
ASTROMETRY OF IMAGE SET'example'
************************************************************


### Telescope and camera (detector) properties

In [4]:
print0("Telescope & CCD Properties:")
if not "detector" in AIA.keys() or CONF.OVERWRITE:
    #Telescope and camera (detector) properties
    detector=dictObj(dict())
    detector.FOCAL=Config(CFG,"FocalLength") #mm
    detector.PW=Config(CFG,"PixelWide") #mm
    detector.PH=Config(CFG,"PixelHigh") #mm
    detector.SIZEX=images[0]["header"]["NAXIS1"]
    detector.SIZEY=images[0]["header"]["NAXIS2"]
    detector.PWD=np.arctan(detector.PW/detector.FOCAL)*RAD
    detector.PHD=np.arctan(detector.PW/detector.FOCAL)*RAD
    detector.PXSIZE=(detector.PWD+detector.PHD)/2
    
    
    AIA["detector"]=detector
    pickle.dump(AIA,open(AIA_FILE,"wb"))
else:
    print("\tDetector properties already extracted")
    AIA=pickle.load(open(AIA_FILE,"rb"))
    detector=AIA["detector"]

print0("\tFocal lenght (mm) :",detector.FOCAL)
print0("\tPixel size (x mm,y mm) :",detector.PW,detector.PH)
print0("\tImage size (x px,y px) :",detector.SIZEX,detector.SIZEY)
print0("\tPixel size (arcsec):",detector.PXSIZE/ARCSEC)
print0("\tCamera field (x deg,y deg) :",detector.SIZEX*detector.PWD,detector.SIZEY*detector.PHD)
print("\tDone.")

Telescope & CCD Properties:
	Focal lenght (mm) : 8070.0
	Pixel size (x mm,y mm) : 0.01 0.01
	Image size (x px,y px) : 2423 2434
	Pixel size (arcsec): 0.255594555449
	Camera field (x deg,y deg) : 0.172029335514 0.172810318878
	Done.


### Astrometry using astromery.net

In [11]:
x="X_IMAGE"
y="Y_IMAGE"
cs=[x,y]

for i,image in enumerate(images):

    file=image["file"]
    header=image["header"]

    # Get bright stars
    imgsources=sources[sources.IMG==i]
    border=borders[i]    
    
    #Choose the first NREFSTARS far from defects
    nbright=0
    indbright=[]
    for ind in imgsources.index:
        dmin=(np.sqrt((imgsources.loc[ind,x]-border[:,0])**2+\
                      (imgsources.loc[ind,y]-border[:,1])**2)).min()
        if dmin<2*CONF.RADIUS:continue
        indbright+=[ind]
        nbright+=1
        if nbright>=CONF.NREFSTARS:break

    bright=imgsources.loc[indbright]
    sources.loc[indbright,"STAR"]=1
    nbright=len(bright)

    print("\t\tNumber of selected bright stars:",len(bright))
    
    # Read fits and replace with bright stars
    img=OUT_DIR+file+".fits"
    hdul=fits.open(img)
    data=hdul[0].data
    data=np.zeros_like(data)
    for ind in bright.index:
        data[int(bright.loc[ind,"X_IMAGE"]),int(bright.loc[ind,"Y_IMAGE"])]=255
    hdul[0].data=data
    hdul.writeto(OUT_DIR+"this-%d.fits"%i,overwrite=True)
    hdul.close()
    
    #GET COORDINATES
    ra=sex2dec(header["OBJCTRA"])*15
    dec=sex2dec(header["OBJCTDEC"])

    #ASTROMETRY.NET COMMAND
    print("Running astrometry over %s..."%file)
    opts=""
    opts+=" "+"--use-sextractor --sextractor-path "+CONF.SEX_DIR+"bin/sex"
    opts+=" "+"--no-plots"
    opts+=" "+"--ra %.7f --dec %.7f --radius 1"%(ra,dec)
    opts+=" "+"--guess-scale --overwrite"
    cmd=CONF.AST_DIR+"bin/solve-field "+opts+" "+"this-%d.fits"%i
    sys="cd "+OUT_DIR+";"+cmd
    out=System(sys,True)
    
    print("\tAstrometry processing successful")
    #break
    
print("Done.")

		Number of selected bright stars: 20
Running astrometry over o8009g0075o.1299949.ch.1991737.XY51.p10...
Reading input file 1 of 1: "this-0.fits"...
Found an existing WCS header, will try to verify it.
Extracting sources...
Sorting brightness using MAG_AUTO and BACKGROUND columns failed; falling back to MAG_AUTO.
Reading sort column "MAG_AUTO"
Sorting sort column
mmapping input file
Copying table header.
Writing row 0
Done
Solving...
Reading file "./this-0.axy"...
Only searching for solutions within 1 degrees of RA,Dec (315.546,-15.5833)
Verifying WCS using indices with quads of size [1.03379, 10.3848] arcmin
Verifying WCS with index 2 of 4 (/home/astrometry/astrometry/data/index-4204-47.fits)
Verifying WCS of field 1.
 --> log-odds 0
Verifying WCS with index 3 of 4 (/home/astrometry/astrometry/data/index-4203-47.fits)
Verifying WCS of field 1.
 --> log-odds 0
Verifying WCS with index 4 of 4 (/home/astrometry/astrometry/data/index-4202-47.fits)
Verifying WCS of field 1.
 --> log-odds 0

Field: this-3.fits
Field center: (RA,Dec) = (316.039785, -15.582531) deg.
Field center: (RA H:M:S, Dec D:M:S) = (21:04:09.548, -15:34:57.111).
Field size: 10.3159 x 10.3927 arcminutes
Field rotation angle: up is 14.6899 degrees E of N
Field parity: pos
Creating new FITS file "./this-3.new"...
	Astrometry processing successful
Done.


In [41]:
print0("Task completed.")
FLOG.close()

Task completed.
